In [ ]:
!pip install hopsworks

In [1]:
import pandas as pd
from datetime import datetime
!python -V
import sys;
print(sys.version)

Python 3.7.15
3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [2]:
# Initialization: Load shared functions and simulated data 
import os 

# Load shared functions
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Get simulated data from Github repository
if not os.path.exists("simulated-data-transformed"):
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-transformed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63257  100 63257    0     0   278k      0 --:--:-- --:--:-- --:--:--  278k
Cloning into 'simulated-data-transformed'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 189 (delta 0), reused 3 (delta 0), pack-reused 186
Receiving objects: 100% (189/189), 70.08 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (184/184), done.


In [3]:
DIR_INPUT='simulated-data-transformed/data/' 

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: user 562 ms, sys: 612 ms, total: 1.17 s
Wall time: 1.2 s
919767 transactions loaded, containing 8195 fraudulent transactions


In [ ]:
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
#!sudo update-alternatives --config python3
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user


In [ ]:
!which -a pip
!conda config --set offline false
!jupyter kernelspec list
sys.path.append('/usr/local/lib/python3.8/site-packages/')
import hopsworks

In [4]:
# Setting the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime(BEGIN_DATE, "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7

print(transactions_df.TX_AMOUNT.value_counts())

(train_df, test_df)=get_train_test_set(transactions_df,start_date_training,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)
print(train_df.TX_AMOUNT.value_counts())
print( train_df.shape)
print( test_df.shape)
# By default, scaling the input data
(train_df, test_df)=scaleData(train_df,test_df,input_features)

print(train_df.TX_AMOUNT.value_counts())

12.50     162
9.00      161
8.61      160
7.91      160
9.52      159
         ... 
297.50      1
413.10      1
213.91      1
178.07      1
200.00      1
Name: TX_AMOUNT, Length: 22401, dtype: int64
10.95     21
10.54     20
9.47      20
6.75      19
7.68      19
          ..
134.92     1
171.91     1
137.26     1
151.71     1
36.15      1
Name: TX_AMOUNT, Length: 14982, dtype: int64
(67180, 23)
(58330, 23)
-1.021093    21
-1.056416    20
-1.030878    20
-1.103912    19
-1.099138    19
             ..
 1.506437     1
 2.561126     1
 0.222385     1
 2.487616     1
-0.419641     1
Name: TX_AMOUNT, Length: 14980, dtype: int64


In [15]:
[x.lower() for x in train_df.columns.to_list()]

['transaction_id',
 'tx_datetime',
 'customer_id',
 'terminal_id',
 'tx_amount',
 'tx_time_seconds',
 'tx_time_days',
 'tx_fraud',
 'tx_fraud_scenario',
 'tx_during_weekend',
 'tx_during_night',
 'customer_id_nb_tx_1day_window',
 'customer_id_avg_amount_1day_window',
 'customer_id_nb_tx_7day_window',
 'customer_id_avg_amount_7day_window',
 'customer_id_nb_tx_30day_window',
 'customer_id_avg_amount_30day_window',
 'terminal_id_nb_tx_1day_window',
 'terminal_id_risk_1day_window',
 'terminal_id_nb_tx_7day_window',
 'terminal_id_risk_7day_window',
 'terminal_id_nb_tx_30day_window',
 'terminal_id_risk_30day_window']

In [ ]:
test_df

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
#transactions_df.to_pickle('dataset.pkl')

train_df[input_features].info()

🛠️ Feature Engineering

In [ ]:
train_df.to_pickle('dataset.pkl')

NameError: ignored

In [ ]:
fraud_labels = transactions_df.copy()[["TRANSACTION_ID", "TX_FRAUD"]]
fraud_labels.to_pickle('dataset_fraud.pkl')